<a href="https://colab.research.google.com/github/ash0411/fiw/blob/main/Copy_of_fiw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-vggface
# to remove some utils files that do not exit 
!pip show keras-vggface
!pip install keras_applications
filename = "/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, pyyaml, h5py, keras, scipy, six, pillow
Required-by: 
     |████████████████████████████████| 50 kB 2.9 MB/s 


20951

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# importing basic libraries
import numpy as np
import pandas as pd
import os
from random import choice, sample
# importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
# importing library for reading image
from PIL import Image
from glob import glob
import tensorflow as tf
from collections import defaultdict
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D,Flatten, Dense, Dropout, InputLayer,GlobalAvgPool2D, GlobalMaxPool2D, Concatenate, Multiply, Subtract, Add
#from tensorflow.keras.layers.pooling import MaxPool2D
#from tensorflow.keras.layers.convolutional import Convolution2D,Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
 # building a model
from keras import backend as K
K.set_image_data_format('channels_last')
import keras
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
import tensorflow as tf
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [4]:
!rmdir /root/.kaggle
!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

rmdir: failed to remove '/root/.kaggle': No such file or directory


In [5]:
!kaggle competitions download -c recognizing-faces-in-the-wild

  0% 0.00/156k [00:00<?, ?B/s]
100% 156k/156k [00:00<00:00, 59.2MB/s]
  0% 0.00/354k [00:00<?, ?B/s]
100% 354k/354k [00:00<00:00, 48.9MB/s]
 85% 58.0M/68.6M [00:00<00:00, 75.5MB/s]
100% 68.6M/68.6M [00:00<00:00, 108MB/s] 
  0% 0.00/77.6k [00:00<?, ?B/s]
100% 77.6k/77.6k [00:00<00:00, 79.3MB/s]
  0% 0.00/1.47k [00:00<?, ?B/s]
100% 1.47k/1.47k [00:00<00:00, 1.33MB/s]
 71% 21.0M/29.7M [00:00<00:00, 62.1MB/s]
100% 29.7M/29.7M [00:00<00:00, 115MB/s] 
  0% 0.00/33.7k [00:00<?, ?B/s]
100% 33.7k/33.7k [00:00<00:00, 30.1MB/s]
 63% 17.0M/27.1M [00:00<00:00, 45.3MB/s]
100% 27.1M/27.1M [00:00<00:00, 76.9MB/s]
 95% 106M/111M [00:00<00:00, 111MB/s] 
100% 111M/111M [00:00<00:00, 126MB/s]
 62% 21.0M/34.1M [00:00<00:00, 46.2MB/s]
100% 34.1M/34.1M [00:00<00:00, 97.8MB/s]
 93% 121M/130M [00:01<00:00, 98.5MB/s]
100% 130M/130M [00:01<00:00, 105MB/s] 
  0% 0.00/8.67k [00:00<?, ?B/s]
100% 8.67k/8.67k [00:00<00:00, 7.50MB/s]


In [6]:
import os
!mkdir face
!unzip 'train.zip' -d '/content/face/train'
!unzip 'test.zip' -d '/content/face/test'

Streaming output truncated to the last 5000 lines.
  inflating: /content/face/test/face04688.jpg  
  inflating: /content/face/test/face03972.jpg  
  inflating: /content/face/test/face02593.jpg  
  inflating: /content/face/test/face00836.jpg  
  inflating: /content/face/test/face05284.jpg  
  inflating: /content/face/test/face03261.jpg  
  inflating: /content/face/test/face00582.jpg  
  inflating: /content/face/test/face02696.jpg  
  inflating: /content/face/test/face05325.jpg  
  inflating: /content/face/test/face04973.jpg  
  inflating: /content/face/test/face02902.jpg  
  inflating: /content/face/test/face06245.jpg  
  inflating: /content/face/test/face00645.jpg  
  inflating: /content/face/test/face03284.jpg  
  inflating: /content/face/test/face00188.jpg  
  inflating: /content/face/test/face05490.jpg  
  inflating: /content/face/test/face01683.jpg  
  inflating: /content/face/test/face05566.jpg  
  inflating: /content/face/test/face03288.jpg  
  inflating: /content/face/test/face0

In [7]:
df = pd.read_csv('train_relationships.csv')

In [8]:
df.head()

,p1,p2
0,F0002/MID1,F0002/MID3
1,F0002/MID2,F0002/MID3
2,F0005/MID1,F0005/MID2
3,F0005/MID3,F0005/MID2
4,F0009/MID1,F0009/MID4


In [9]:
# finding the number of families in training and putting them in a list
train_url =  '/content/face/train/'
family = os.listdir(train_url)
print('the number of families are {}'.format(len(family)))
print(family[:10])

the number of families are 470
['F0167', 'F0794', 'F0607', 'F0758', 'F0249', 'F0982', 'F0178', 'F0679', 'F0752', 'F0838']


In [10]:
#finding the number of families in the test case and putting them in a list
# finding the number of families in training and putting them in a list
test_url =  '/content/face/test/'
family_pics = os.listdir(test_url)
print("the number of testing pictures are {}".format(len(family_pics)))
print(family_pics[:10])

the number of testing pictures are 6282
['face02540.jpg', 'face03076.jpg', 'face00904.jpg', 'face00150.jpg', 'face01010.jpg', 'face00928.jpg', 'face04467.jpg', 'face05772.jpg', 'face00671.jpg', 'face01924.jpg']


In [11]:
train_file_path = "train_relationships.csv"
val_famillies = "F09"

In [12]:
# seperating the destination of training and testing images
all_images = glob(train_url + "*/*/*.jpg")
train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

In [13]:
train_person_to_images_map = defaultdict(list)
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]
training_families = []
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    if x.split('/')[-3] in training_families:
      continue
    else:
      training_families.append(x.split('/')[-3])

val_person_to_images_map = defaultdict(list)
val_families = []
for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    if x.split('/')[-3]  in val_families:
      continue
    else:
      val_families.append(x.split('/')[-3])
# to seperate images based on their family and number within family
  

In [14]:
len(val_families)

50

In [15]:
df = list(zip(df.p1.values, df.p2.values))
df = [x for x in df if x[0] in ppl and x[1] in ppl]

In [16]:
train = [x for x in df if val_famillies not in x[0]]
val = [x for x in df if val_famillies in x[0]]

In [17]:
for x in train:
  if 'F0290' in x[0]:
    print(x) 

('F0290/MID1', 'F0290/MID6')
('F0290/MID1', 'F0290/MID3')
('F0290/MID1', 'F0290/MID4')
('F0290/MID1', 'F0290/MID2')
('F0290/MID2', 'F0290/MID3')
('F0290/MID2', 'F0290/MID4')
('F0290/MID2', 'F0290/MID6')
('F0290/MID3', 'F0290/MID6')
('F0290/MID3', 'F0290/MID4')
('F0290/MID4', 'F0290/MID6')
('F0290/MID5', 'F0290/MID3')
('F0290/MID5', 'F0290/MID2')
('F0290/MID5', 'F0290/MID4')


In [18]:
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    #plt.imshow(img)
    img = np.array(img).astype(np.float)#reshape(3,224,224)
    #print(img.shape)
    #img = np.array([img])
    #print(img.shape)
    return img

In [19]:

#read_img(train_person_to_images_map[train[0][0]][0])

In [20]:
ppl = list(train_person_to_images_map.keys())

In [24]:
print(ppl)

['F0179/MID4', 'F0179/MID1', 'F0179/MID2', 'F0179/MID3', 'F0739/MID4', 'F0739/MID1', 'F0739/MID5', 'F0739/MID2', 'F0739/MID3', 'F0063/MID1', 'F0063/MID3', 'F0411/MID1', 'F0411/MID2', 'F0411/MID3', 'F0568/MID4', 'F0568/MID1', 'F0568/MID5', 'F0568/MID2', 'F0568/MID3', 'F0826/MID4', 'F0826/MID1', 'F0826/MID2', 'F0826/MID3', 'F0482/MID1', 'F0482/MID3', 'F0746/MID4', 'F0746/MID1', 'F0746/MID2', 'F0746/MID3', 'F0049/MID7', 'F0049/MID4', 'F0049/MID6', 'F0049/MID5', 'F0049/MID3', 'F0041/MID4', 'F0041/MID6', 'F0041/MID1', 'F0579/MID7', 'F0579/MID4', 'F0579/MID6', 'F0579/MID8', 'F0579/MID1', 'F0579/MID5', 'F0579/MID2', 'F0579/MID9', 'F0579/MID3', 'F0769/MID7', 'F0769/MID4', 'F0769/MID6', 'F0769/MID1', 'F0769/MID5', 'F0769/MID2', 'F0769/MID3', 'F0500/MID4', 'F0500/MID1', 'F0500/MID5', 'F0500/MID2', 'F0500/MID3', 'F0233/MID6', 'F0233/MID1', 'F0233/MID5', 'F0233/MID2', 'F0233/MID3', 'F0726/MID4', 'F0726/MID1', 'F0726/MID2', 'F0726/MID3', 'F0119/MID1', 'F0119/MID2', 'F0119/MID3', 'F0705/MID4', 'F070

In [33]:
"""
batch_size = 16
ppl = list(train_person_to_images_map.keys())
while True:
    batch_tuples = sample(train, batch_size // 2)
    labels = [1] * len(batch_tuples)
    while len(batch_tuples) < batch_size:
        p1 = choice(ppl)
        p2 = choice(ppl)
        # to append same no of relations that are in the table to no of relations not in the relatios
        # to make the data have same proportion of negative and postive examples
        if p1 != p2 and (p1, p2) not in train and (p2, p1) not in train:
            batch_tuples.append((p1, p2))
            labels.append(0)

    for x in batch_tuples:
        if not len(train_person_to_images_map[x[0]]):
            print(x[0])
    print(batch_tuples)
    # transfering the first set of images in list X1
    # selecting random images for a particular person from the list
    X1 = [choice(train_person_to_images_map[x[0]]) for x in batch_tuples]
    print(X1)
    X1 = np.array([read_img(x) for x in X1])
    # transfering the second set of images in list X2
    # selecting random sets of images for a particular person in the list
    X2 = [choice(train_person_to_images_map[x[1]]) for x in batch_tuples]
    X2 = np.array([read_img(x) for x in X2])
    print(labels)
    return X1, X2, labels
"""

'\nbatch_size = 16\nppl = list(train_person_to_images_map.keys())\nwhile True:\n    batch_tuples = sample(train, batch_size // 2)\n    labels = [1] * len(batch_tuples)\n    while len(batch_tuples) < batch_size:\n        p1 = choice(ppl)\n        p2 = choice(ppl)\n        # to append same no of relations that are in the table to no of relations not in the relatios\n        # to make the data have same proportion of negative and postive examples\n        if p1 != p2 and (p1, p2) not in train and (p2, p1) not in train:\n            batch_tuples.append((p1, p2))\n            labels.append(0)\n\n    for x in batch_tuples:\n        if not len(train_person_to_images_map[x[0]]):\n            print(x[0])\n    print(batch_tuples)\n    # transfering the first set of images in list X1\n    # selecting random images for a particular person from the list\n    X1 = [choice(train_person_to_images_map[x[0]]) for x in batch_tuples]\n    print(X1)\n    X1 = np.array([read_img(x) for x in X1])\n    # tran

In [34]:
a = [(x,y) for x,y in train if 'F0290' in x]
c = [(x,y) for x,y in train if 'F0179' in x]
b = set([x for x,y in train if 'F0290' in x])
d = set([x for x,y in train if 'F0179' in x])

In [ ]:
e = a + c

In [ ]:
#e

In [ ]:
num = len(a) + len(c)
labels =  [1] * num
print('above while',num)
print(len(e))
while len(e) <= 2 * num:
  #print('into while')
  person1 = choice(list(b))
  person2 = choice(list(d))
  e.append((person1, person2))
  labels.append(0)
  for x in train:
    if not len(train_person_to_images_map[x[0]]):
        print(x[0])
X1 = [choice(train_person_to_images_map[x[0]]) for x in e]
X1 = np.array([read_img(x) for x in X1])
# transfering the second set of images in list X2
# selecting random sets of images for a particular person in the list
X2 = [choice(train_person_to_images_map[x[1]]) for x in e]
X2 = np.array([read_img(x) for x in X2])
print(labels)
tr = [[x1,x2,label] for x1 in X1 for x2 in X2 for label in labels]
df1 = pd.DataFrame(tr, columns = ['image1','image2','labels'])
#dict = {'image1': X1, 'image':X2, 'labels': labels}
#df1.append(X1,X2,labels)

above while 18
18
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
df1.describe()

,labels
count,50653.000000
mean,0.486486
std,0.499822
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [ ]:
#e

,image1,image2,labels


In [19]:
def gen2(data, person_to_images_map,training_families):
  df1 = pd.DataFrame(columns = ['image1','image2','labels'])
  for i in range(len(training_families)):
    for j in range(i+1,len(training_families)):
      family_a = [(x,y) for x,y in data if training_families[i] in x] # list of families for 1st family
      family_set_a = set([x for x,y in data if training_families[i] in x])# set of families for 1st family
      family_b = [(x,y) for x,y in data if training_families[j] in x] 
      family_set_b = set([x for x,y in data if training_families[j] in x])
      families = family_a + family_b
      family_combination_count = len(family_a) + len(family_b)
      labels = [1] * family_combination_count
      while len(families) <= 2 * family_combination_count: 
        #print('family_set_a',family_set_a)
        #print('family_set_b',family_set_b)
        #print('family name set b', training_families[j])
        person1 = choice(list(family_set_a))
        person2 = choice(list(family_set_b))
        #if person1 != person2 and (person1, person2) not in families and (person2, person1) not in families:
        families.append((person1, person2))
        labels.append(0)
      for x in data:
          if not len(person_to_images_map[x[0]]):
              print(x[0])
      # transfering the first set of images in list X1
      # selecting random images for a particular person from the list 
      X1 = [choice(person_to_images_map[x[0]]) for x in families]
      X1 = np.array([read_img(x) for x in X1])
      # transfering the second set of images in list X2
      # selecting random sets of images for a particular person in the list
      X2 = [choice(person_to_images_map[x[1]]) for x in families]
      X2 = np.array([read_img(x) for x in X2])
      #print(labels)
      tr = [[x1,x2,label] for x1 in X1 for x2 in X2 for label in labels] 
      df2 = pd.DataFrame(tr, columns = ['image1','image2','labels'])
      df1.append(df2)
  return df1

In [20]:
#training_families.remove('F0482')
#training_families.remove('F0524')
#training_families.remove('F0691')
#training_families.remove('F0275')
#training_families.remove('F0744')
#training_families.remove('F0253')
#training_families.remove('F0244')
training_families.remove('F0452')

In [21]:
gen2(train,train_person_to_images_map,training_families)

IndexError: ignored

In [21]:
def gen(list_tuples, person_to_images_map, batch_size=32):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)
        # to append same no of relations that are in the table to no of relations not in the relations
        # to make the data have same proportion of negative and postive examples

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])
        # transfering the first set of images in list X1
        # selecting random images for a particular person from the list 
        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])
        # transfering the second set of images in list X2
        # selecting random sets of images for a particular person in the list
        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])
        #print(labels)
        yield [np.array(X1), np.array(X2)], np.array(labels)

In [22]:
temp = gen(train,train_person_to_images_map)

In [23]:
base_model = VGGFace(model = 'resnet50', include_top = False)
base_model.summary()

94707712/94694792 [==============================] - 2s 0us/step
Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, None, None,   256        ['conv1/7x7_s2[0][0]']           
 ation)           

In [24]:
# vgg_facenet = VGGFace(model ='resnet50', include_top = False)
# output = vgg_facenet.layers[-2].output
# #output = keras.layers.Flatten()(output)
# vgg_facenet = Model(vgg_facenet.input, outputs=output)
# for layer in vgg_facenet.layers:
#     layer.trainable = False
# vgg_facenet.summary()
def baseline_model():
    input_1 = Input(shape=(224,224,3))
    input_2 = Input(shape=(224,224,3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalAvgPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalAvgPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4  = Subtract()([x1_, x2_])    
    x   = Concatenate(axis=-1)([x4, x3])
    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x) 
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = BatchNormalization()(x)
    x   = Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x   = Dropout(0.01)(x)    
    x   = Dense(100, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)    
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
    model.summary()

    return model

In [ ]:
file_path = "/drive/MyDrive/vgg_face.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, 
                          patience=15, verbose=0, mode='auto')
reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.2, 
                                      patience=10, verbose=1)

callbacks_list = [checkpoint,early_stop]

curr_model = baseline_model()
curr_model_hist=curr_model.fit(gen(train, train_person_to_images_map, batch_size=32), 
                            use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=32), 
                            epochs=10, 
                            verbose=1,workers = 4, 
                            callbacks=callbacks_list,
                            steps_per_epoch=200,
                            validation_steps=100)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vggface_resnet50 (Functional)  (None, None, None,   23561152    ['input_2[0][0]',                
                                2048)                             'input_3[0][0]']            

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


200/200 [==============================] - 332s 2s/step - loss: 12.2076 - acc: 0.5639 - val_loss: 12.1065 - val_acc: 0.5869
Epoch 2/10
200/200 [==============================] - ETA: 0s - loss: 11.9892 - acc: 0.6311
Epoch 00002: val_acc improved from 0.58688 to 0.62437, saving model to /drive/MyDrive/vgg_face.h5
200/200 [==============================] - 307s 2s/step - loss: 11.9892 - acc: 0.6311 - val_loss: 11.9023 - val_acc: 0.6244
Epoch 3/10
200/200 [==============================] - ETA: 0s - loss: 11.7802 - acc: 0.6689
Epoch 00003: val_acc improved from 0.62437 to 0.65625, saving model to /drive/MyDrive/vgg_face.h5
200/200 [==============================] - 310s 2s/step - loss: 11.7802 - acc: 0.6689 - val_loss: 11.7097 - val_acc: 0.6562
Epoch 4/10
200/200 [==============================] - ETA: 0s - loss: 11.5616 - acc: 0.7094

In [ ]:

 train

In [ ]:
curr_model.save('/drive/model1.h5')